In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#import modules
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np
import plotly.express as px

import os
import glob

from multiprocessing import Pool

from pandarallel import pandarallel
pandarallel.initialize()

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

import time
import matplotlib.pyplot as plt
%matplotlib inline

path_to_files = "../input/optiver-realized-volatility-prediction"

book_train_files =  path_to_files + '/book_train.parquet/stock_id={}'
trade_train_files =  path_to_files + '/trade_train.parquet/stock_id={}'

book_test_files =  path_to_files + '/book_test.parquet/stock_id={}'
trade_test_files =  path_to_files + '/trade_test.parquet/stock_id={}'

SMALL_F = 0.00000001

In [ ]:
!pip -q install ../input/pytorchtabnet/pytorch_tabnet-3.1.1-py3-none-any.whl

In [ ]:
import os
import glob
from joblib import Parallel, delayed
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
import lightgbm as lgb
import warnings
import random
from collections import Counter, defaultdict
from sklearn.cluster import KMeans
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer, LabelEncoder, StandardScaler
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
import gc

import optuna.integration.lightgbm as lgbo
import numpy.matlib

from sklearn.model_selection import train_test_split

from sklearn.decomposition import FastICA, PCA

import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator

from scipy import stats
from scipy.stats import norm
from joblib import Parallel, delayed

import shutil

from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetRegressor

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts

# setting some globl config

plt.style.use('ggplot')
orange_black = [
    '#fdc029', '#df861d', '#FF6347', '#aa3d01', '#a30e15', '#800000', '#171820'
]
plt.rcParams['figure.figsize'] = (16,9)
plt.rcParams["figure.facecolor"] = '#FFFACD'
plt.rcParams["axes.facecolor"] = '#FFFFE0'
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = orange_black[3]
plt.rcParams["grid.alpha"] = 0.5
plt.rcParams["grid.linestyle"] = '--'

warnings.filterwarnings('ignore')

In [ ]:
from numpy.random import seed
import tensorflow as tf
from tensorflow import keras
import numpy as np
from keras import backend as K
from keras.engine.topology import Layer

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

In [ ]:
#Configuration
cfg = dict(
    isCollectDataOnly = True,
    isStockIdUsed = False,
    isTFModelUsed = False,
    trainNotUsedCols = ['row_id', 'target', 'time_id', 'stock_id'],
    predictNotUsedCols = ['row_id', 'time_id', 'stock_id'],
    useHyperOpt = False,
    useLabelTransformation = False,
    volumeBarThreshold = 500.0
)


cfg

## Useful functions

In [ ]:
# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

def log_return(prob):
    prob += SMALL_F
    return np.log(prob).diff()

def log_return2(x):
    return np.log1p(x.pct_change())

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))


def getOscStoch(x):
    return (x[-1] - np.min(x))/(np.max(x)-np.min(x)+SMALL_F)

def getNormVal(x):
    return (x[-1])/(np.mean(x)+SMALL_F)

def getBreath(ret, size):
    f = ret > 0
    upside = size[f]
    downside = size[~f]
    
    di = np.sum(upside)/(np.sum(downside)+SMALL_F)
    ado = np.sum(upside) - np.sum(downside)
    
    return di, ado
    
def rateLastFirst(x):
    return np.mean(x)/(np.sum(x)+SMALL_F)

def rolling_windows_vectorized(array, sub_window_size=2):
    start = 0
    max_time = len(array)-sub_window_size
    sub_windows = (
        start +
        # expand_dims are used to convert a 1D array to 2D array.
        np.expand_dims(np.arange(sub_window_size), 0) +
        np.expand_dims(np.arange(max_time + 1), 0).T
    )
    
    return array[sub_windows]

def rolling_windows_vectorized_v2(array, sub_window_size, stride_size):
    start = 0
    max_time = len(array)-sub_window_size
    sub_windows = (
        start + 
        np.expand_dims(np.arange(sub_window_size), 0) +
        # Create a rightmost vector as [0, V, 2V, ...].
        np.expand_dims(np.arange(max_time + 1, step=stride_size), 0).T
    )
    
    return array[sub_windows]    

def getVolumaBars(i_data, threshold=1000.0):
    o, h, l, c, v  = 0.0, 0.0, 1000000.0, 0.0, 0.0
    res_array = []
    isNewBar = True
    bar_index = 0.0
    cum_volume = 0.0
    data_len = i_data.shape[0]
    for i in range(data_len):
        #print(i_data[i])
        cur_price = i_data[i][0]
        #print(cur_price)
        if True == isNewBar:
            bar_index = i
            o = cur_price
            c = 0.0
            h = 0.0
            l = 10000000.0
            v = 0.0
            
            isNewBar = False

        if cur_price > h:
            h = cur_price
        if cur_price < l:
            l = cur_price

        v += i_data[i][1]
        
        if (v >= threshold) or (i == data_len-1):
            isNewBar = True
            #bar_index = i
            c = cur_price
            res_array.append([bar_index, o, h, l, c, v])


    return pd.DataFrame(res_array, columns=['bar_index', 'open', 'high', 'low', 'close', 'volume'])


def _get_beta(high, low, window):

    beta_r = np.empty(high.shape)
    beta_r[:] = np.NaN

    ret = np.log(high / low)
    high_low_ret = ret ** 2
    beta = rolling_windows_vectorized(high_low_ret, 2).sum(axis=1)
    beta = rolling_windows_vectorized(beta, window).mean(axis=1)
    beta_r[len(beta_r)-len(beta):] = beta    

    return beta_r


def _get_gamma(high, low):
    gamma_r = np.empty(high.shape)
    gamma_r[:] = np.NaN

    high_max = rolling_windows_vectorized(high, 2).max(axis=1)
    low_min = rolling_windows_vectorized(low, 2).min(axis=1)
    gamma = np.log(high_max / low_min) ** 2
    gamma_r[len(gamma_r)-len(gamma):] = gamma

    return gamma_r    

def get_bekker_parkinson_vol2(high, low, window: int = 20):

    beta = _get_beta(high, low, window)
    gamma = _get_gamma(high, low)

    k2 = (8 / np.pi) ** 0.5
    den = 3 - 2 * 2 ** .5
    sigma = (2 ** -0.5 - 1) * beta ** 0.5 / (k2 * den)
    sigma += (gamma / (k2 ** 2 * den)) ** 0.5
    sigma[sigma < 0] = 0

    return sigma

def get_garman_class_vol2(open, high, low, close, window):
    ret_value = np.empty(high.shape)
    ret_value[:] = np.NaN
    ret = np.log(high / low)  # High/Low return
    close_open_ret = np.log(close / open)  # Close/Open return
    estimator = 0.5 * ret ** 2 - (2 * np.log(2) - 1) * close_open_ret ** 2
    ret_v = rolling_windows_vectorized(estimator, window).mean(axis=1)
    ret_value[len(ret_value)-len(ret_v):] = np.sqrt(ret_v)
    return ret_value

def getWindows(n_bars, isSpecialWindow=False, min_bar_length=3):
    if(True==isSpecialWindow):
        window_size = n_bars//2
    else:
        window_size = n_bars-(min_bar_length-1)

    if(window_size<=0):
        if(n_bars-1 > 0):
            window_size = n_bars - 1
        else:
            window_size=1

    return window_size

def getMicrostructuralFeatures(input_df, output_df, col_prefix = '', col_postfix = '', min_bar_length = 3, volumeThreshold = cfg['volumeBarThreshold'], isParkinson=True, isGarman=True, isYyang=True, isBekker=True, isMicro=True, isSpecialWindow=False, micro_cols=[]):
    
    v_sum = np.sum(input_df.loc[:, 'size']).astype(np.float64)
    thres = volumeThreshold if v_sum >= volumeThreshold else v_sum
    volume_bars = getVolumaBars(input_df.loc[:,['price','size']].to_numpy(), thres)

    window_size = getWindows(len(volume_bars), isSpecialWindow=isSpecialWindow, min_bar_length=min_bar_length)

    if(True == isGarman):
        col_name = col_prefix+'garman_class_vol'+col_postfix
        pv = get_garman_class_vol2(volume_bars.open.to_numpy(), volume_bars.high.to_numpy(), volume_bars.low.to_numpy(), volume_bars.close.to_numpy(), window=window_size)
        pv = pv[~np.isnan(pv)]
        if(len(pv)>0):
            output_df.loc[:,col_name] = np.median(pv)
        else:
            output_df.loc[:,col_name] = 0.0

    if(True == isBekker):
        col_name = col_prefix+'bekker_parkinson_vol'+col_postfix
        pv = get_bekker_parkinson_vol2(volume_bars.high.to_numpy(), volume_bars.low.to_numpy(), window=window_size)
        pv = pv[~np.isnan(pv)]
        if(len(pv)>0):
            output_df.loc[:,col_name] = np.median(pv)
        else:
            output_df.loc[:,col_name] = 0.0

    return output_df

def sqsum(series):
    return np.sum(np.square(series))

# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return_new(series):
    return np.log(series).diff()

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

def realized_quarticity(series):
    return np.sum(series**4)*series.shape[0]/3

def realized_1(series):
    return np.sqrt(np.sum(series**4)/(6*np.sum(series**2)))

def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0, 600)], names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

def fix_offsets(data_df):
    offsets = data_df.groupby(['time_id']).agg({'seconds_in_bucket':'min'})
    offsets.columns = ['offset']
    data_df = data_df.join(offsets, on='time_id')
    data_df.seconds_in_bucket = data_df.seconds_in_bucket - data_df.offset
    return data_df

In [ ]:
# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

def log_return(prob):
    prob += SMALL_F
    return np.log(prob).diff()

def log_return2(x):
    return np.log1p(x.pct_change())

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))


def getOscStoch(x):
    return (x[-1] - np.min(x))/(np.max(x)-np.min(x)+SMALL_F)

def getNormVal(x):
    return (x[-1])/(np.mean(x)+SMALL_F)

def getBreath(ret, size):
    f = ret > 0
    upside = size[f]
    downside = size[~f]
    
    di = np.sum(upside)/(np.sum(downside)+SMALL_F)
    ado = np.sum(upside) - np.sum(downside)
    
    return di, ado
    
def rateLastFirst(x):
    return np.mean(x)/(np.sum(x)+SMALL_F)

def rolling_windows_vectorized(array, sub_window_size=2):
    start = 0
    max_time = len(array)-sub_window_size
    sub_windows = (
        start +
        # expand_dims are used to convert a 1D array to 2D array.
        np.expand_dims(np.arange(sub_window_size), 0) +
        np.expand_dims(np.arange(max_time + 1), 0).T
    )
    
    return array[sub_windows]

def rolling_windows_vectorized_v2(array, sub_window_size, stride_size):
    start = 0
    max_time = len(array)-sub_window_size
    sub_windows = (
        start + 
        np.expand_dims(np.arange(sub_window_size), 0) +
        # Create a rightmost vector as [0, V, 2V, ...].
        np.expand_dims(np.arange(max_time + 1, step=stride_size), 0).T
    )
    
    return array[sub_windows]    

def getVolumaBars(i_data, threshold=1000.0):
    o, h, l, c, v  = 0.0, 0.0, 1000000.0, 0.0, 0.0
    res_array = []
    isNewBar = True
    bar_index = 0.0
    cum_volume = 0.0
    data_len = i_data.shape[0]
    for i in range(data_len):
        #print(i_data[i])
        cur_price = i_data[i][0]
        #print(cur_price)
        if True == isNewBar:
            bar_index = i
            o = cur_price
            c = 0.0
            h = 0.0
            l = 10000000.0
            v = 0.0
            
            isNewBar = False

        if cur_price > h:
            h = cur_price
        if cur_price < l:
            l = cur_price

        v += i_data[i][1]
        
        if (v >= threshold) or (i == data_len-1):
            isNewBar = True
            #bar_index = i
            c = cur_price
            res_array.append([bar_index, o, h, l, c, v])


    return pd.DataFrame(res_array, columns=['bar_index', 'open', 'high', 'low', 'close', 'volume'])


def _get_beta(high, low, window):

    beta_r = np.empty(high.shape)
    beta_r[:] = np.NaN

    ret = np.log(high / low)
    high_low_ret = ret ** 2
    beta = rolling_windows_vectorized(high_low_ret, 2).sum(axis=1)
    beta = rolling_windows_vectorized(beta, window).mean(axis=1)
    beta_r[len(beta_r)-len(beta):] = beta    

    return beta_r


def _get_gamma(high, low):
    gamma_r = np.empty(high.shape)
    gamma_r[:] = np.NaN

    high_max = rolling_windows_vectorized(high, 2).max(axis=1)
    low_min = rolling_windows_vectorized(low, 2).min(axis=1)
    gamma = np.log(high_max / low_min) ** 2
    gamma_r[len(gamma_r)-len(gamma):] = gamma

    return gamma_r    

def get_bekker_parkinson_vol2(high, low, window: int = 20):

    beta = _get_beta(high, low, window)
    gamma = _get_gamma(high, low)

    k2 = (8 / np.pi) ** 0.5
    den = 3 - 2 * 2 ** .5
    sigma = (2 ** -0.5 - 1) * beta ** 0.5 / (k2 * den)
    sigma += (gamma / (k2 ** 2 * den)) ** 0.5
    sigma[sigma < 0] = 0

    return sigma

def get_garman_class_vol2(open, high, low, close, window):
    ret_value = np.empty(high.shape)
    ret_value[:] = np.NaN
    ret = np.log(high / low)  # High/Low return
    close_open_ret = np.log(close / open)  # Close/Open return
    estimator = 0.5 * ret ** 2 - (2 * np.log(2) - 1) * close_open_ret ** 2
    ret_v = rolling_windows_vectorized(estimator, window).mean(axis=1)
    ret_value[len(ret_value)-len(ret_v):] = np.sqrt(ret_v)
    return ret_value

def getWindows(n_bars, isSpecialWindow=False, min_bar_length=3):
    if(True==isSpecialWindow):
        window_size = n_bars//2
    else:
        window_size = n_bars-(min_bar_length-1)

    if(window_size<=0):
        if(n_bars-1 > 0):
            window_size = n_bars - 1
        else:
            window_size=1

    return window_size

def getMicrostructuralFeatures(input_df, output_df, col_prefix = '', col_postfix = '', min_bar_length = 3, volumeThreshold = cfg['volumeBarThreshold'], isParkinson=True, isGarman=True, isYyang=True, isBekker=True, isMicro=True, isSpecialWindow=False, micro_cols=[]):
    
    v_sum = np.sum(input_df.loc[:, 'size']).astype(np.float64)
    thres = volumeThreshold if v_sum >= volumeThreshold else v_sum
    volume_bars = getVolumaBars(input_df.loc[:,['price','size']].to_numpy(), thres)

    window_size = getWindows(len(volume_bars), isSpecialWindow=isSpecialWindow, min_bar_length=min_bar_length)

    if(True == isGarman):
        col_name = col_prefix+'garman_class_vol'+col_postfix
        pv = get_garman_class_vol2(volume_bars.open.to_numpy(), volume_bars.high.to_numpy(), volume_bars.low.to_numpy(), volume_bars.close.to_numpy(), window=window_size)
        pv = pv[~np.isnan(pv)]
        if(len(pv)>0):
            output_df.loc[:,col_name] = np.median(pv)
        else:
            output_df.loc[:,col_name] = 0.0

    if(True == isBekker):
        col_name = col_prefix+'bekker_parkinson_vol'+col_postfix
        pv = get_bekker_parkinson_vol2(volume_bars.high.to_numpy(), volume_bars.low.to_numpy(), window=window_size)
        pv = pv[~np.isnan(pv)]
        if(len(pv)>0):
            output_df.loc[:,col_name] = np.median(pv)
        else:
            output_df.loc[:,col_name] = 0.0

    return output_df

def sqsum(series):
    return np.sum(np.square(series))

# Function to calculate first WAP
def calc_wap1(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

# Function to calculate second WAP
def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def calc_wap3(df):
    wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap4(df):
    wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

# Function to calculate the log of the return
# Remember that logb(x / y) = logb(x) - logb(y)
def log_return_new(series):
    return np.log(series).diff()

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

def realized_quarticity(series):
    return np.sum(series**4)*series.shape[0]/3

def realized_1(series):
    return np.sqrt(np.sum(series**4)/(6*np.sum(series**2)))

def ffill(data_df):
    data_df=data_df.set_index(['time_id', 'seconds_in_bucket'])
    data_df = data_df.reindex(pd.MultiIndex.from_product([data_df.index.levels[0], np.arange(0, 600)], names = ['time_id', 'seconds_in_bucket']), method='ffill')
    return data_df.reset_index()

def fix_offsets(data_df):
    offsets = data_df.groupby(['time_id']).agg({'seconds_in_bucket':'min'})
    offsets.columns = ['offset']
    data_df = data_df.join(offsets, on='time_id')
    data_df.seconds_in_bucket = data_df.seconds_in_bucket - data_df.offset
    return data_df
def getDataFromBidAsk_numpy(df, ci):
    a = 0
    b = 0
    spread  = {}
    for k in [1,2]:
        #k = i+1
        bidp = 'bid_price{}'.format(k)
        askp = 'ask_price{}'.format(k)
        bids = 'bid_size{}'.format(k)
        asks = 'ask_size{}'.format(k)
        #calculate comulative wap
        a += (df[:,ci[bidp]] * df[:,ci[asks]] + df[:,ci[askp]] * df[:,ci[bids]])
        b += df[:,ci[bids]] + df[:,ci[asks]]

        #wap 1 and 2
        spread[f'fb_w_{k}'] = (df[:,ci[bidp]] * df[:,ci[asks]] + df[:,ci[askp]] * df[:,ci[bids]] ) / (df[:,ci[bids]] + df[:,ci[asks]] + SMALL_F)
        spread[f'fb_mid_point_{k}'] = (df[:,ci[askp]]) + (df[:,ci[bidp]]) / 2
        spread[f'fb_volume_total_{k}'] = (df[:,ci[asks]]) + (df[:,ci[bids]])

    
    # mean wap
    spread['fb_w'] = (a/(b+SMALL_F))
    # rates
    spread['fb_w_rate'] = (spread['fb_w_1']) / (spread['fb_w_2']+SMALL_F) 
    spread['fb_mid_point_rate'] = (spread['fb_mid_point_1']) / (spread['fb_mid_point_2']+SMALL_F)
    #sum volume
    spread['fb_volume_total'] = spread['fb_volume_total_1'] + spread['fb_volume_total_2']
    
    ################# test ##################
    spread['ask_1'] = df[:,ci['ask_price1']]
    spread['bid_1'] = df[:,ci['bid_price1']]
    spread['ask_2'] = df[:,ci['ask_price2']]
    spread['bid_2'] = df[:,ci['bid_price2']]
    #########################################
    
    return spread



def Fx(group, stock_id=0, n=10):
    new_df = pd.DataFrame()
    name = int(group.time_id.unique()[0])
    tmp = pd.DataFrame()

    #calculate log return from the following features:
    cols = [
        'fb_w', 
        'fb_w_1', 
        'fb_w_2',
        'fb_mid_point_1',
        'fb_mid_point_rate',
        'fb_w_rate',
    ]

    new_cols = [s + '_lr' for s in cols]
    group.loc[:,new_cols] = log_return2(group[cols]).to_numpy()
    group = group[~group['fb_w'].isnull()]
    
    #calculate realized volatility
    cols = [
        'fb_mid_point_1_lr',
        'fb_mid_point_rate_lr',
        'fb_w_rate_lr',
    ]
    new_cols = [s + '_vola' for s in cols]
    tmp = pd.concat([tmp, pd.DataFrame(realized_volatility(group.loc[:,cols]).to_numpy().reshape(1,-1), columns=new_cols)], axis=1)

    #calculate sum of log return
    new_cols = [s + '_sum' for s in cols]
    tmp = pd.concat([tmp, pd.DataFrame(np.sum(group.loc[:,cols]).to_numpy().reshape(1,-1), columns=new_cols)], axis=1)

    #calclulate market microstructural features
    c = '1'
    cols_1 = []
    new_df = pd.DataFrame({'price': group.loc[:,'fb_mid_point_'+c].to_numpy().flatten(), 'size': group.loc[:,'fb_volume_total_'+c].to_numpy().flatten()}).reset_index()
    tmp = getMicrostructuralFeatures(new_df, tmp, col_prefix = 'fb_', col_postfix = '_'+c, micro_cols=cols_1)
    c = '2'
    cols_2 = []  
    new_df = pd.DataFrame({'price': group.loc[:,'fb_mid_point_'+c].to_numpy().flatten(), 'size': group.loc[:,'fb_volume_total_'+c].to_numpy().flatten()}).reset_index()
    tmp = getMicrostructuralFeatures(new_df, tmp, col_prefix = 'fb_', col_postfix = '_'+c, micro_cols=cols_2)
    
    ############ test idea ################
    col_name = "Test_1"
    window_size = getWindows(len(group), isSpecialWindow=False, min_bar_length=3)
    pv = get_bekker_parkinson_vol2(group.ask_1.to_numpy(), group.bid_1.to_numpy(), window=window_size)
    pv = pv[~np.isnan(pv)]
    if(len(pv)>0):
        tmp.loc[:,col_name] = np.median(pv)
    else:
        tmp.loc[:,col_name] = 0.0

    col_name = "Test_2"
    window_size = getWindows(len(group), isSpecialWindow=False, min_bar_length=3)
    pv = get_bekker_parkinson_vol2(group.ask_2.to_numpy(), group.bid_2.to_numpy(), window=window_size)
    pv = pv[~np.isnan(pv)]
    if(len(pv)>0):
        tmp.loc[:,col_name] = np.median(pv)
    else:
        tmp.loc[:,col_name] = 0.0
    #######################################
    
    tmp.loc[:,'row_id'] = str(stock_id) + '-' + str(name)
    #tmp.loc[:,'time_id'] = int(name)
    return tmp
def getFeaturesFromBookData(df, stock_id, n=10):
    results = df.groupby(['time_id']).apply(Fx, stock_id=stock_id, n=n).reset_index(drop=True)
    return results

## Constract \[stock_id,time_id\] features


In [ ]:
def getDataFromTrade(df):
    log_ret = log_return(df.price).dropna()
    rz_vol = realized_volatility(log_ret)
    
    tmp = pd.DataFrame()
    
    tmp.loc[:,'p_lr_rate'] = [rateLastFirst(log_ret)]
    
    tmp.loc[:,'p_price_count'] = count_unique(df['price'].to_numpy())
    tmp.loc[:,'p_sec_count'] = count_unique(df['seconds_in_bucket'].to_numpy())

    cols_p = []  
    tmp = getMicrostructuralFeatures(df.loc[:, ['price', 'size']], tmp, col_prefix = 'p_', col_postfix = '', isParkinson=False, isGarman=False, isYyang=False, micro_cols=cols_p)

    tmp.loc[:,'p_size_mean'] = np.mean(df['size']) 

    time_id = df.time_id.unique()[0]
    tmp.loc[:,'time_id'] = time_id
    return tmp

def getFeaturesFromTradeData(df):
    return df.groupby(['time_id']).apply(getDataFromTrade).reset_index(drop=True)

# add data

## collect and preprocess data

In [ ]:
# data directory
data_dir = '../input/optiver-realized-volatility-prediction/'

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

def calc_price(df):
    diff = abs(df.filter(like='price').diff())
    min_diff = np.nanmin(diff.where(lambda x: x > 0))
    n_ticks = (diff / min_diff).round()
    return 0.01 / np.nanmean(diff / n_ticks)

# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    stock_id = file_path.split('=')[1]
    spread = getDataFromBidAsk_numpy(df.to_numpy(),{k: v for v, k in enumerate(df.columns.values)})
    df_book_data = pd.concat([df, pd.DataFrame(spread)], axis=1)
    df_book_datar = getFeaturesFromBookData(df_book_data, stock_id, 10)
    df_prices = df.groupby(['time_id']).apply(calc_price).to_frame('prices').reset_index()
    
    df_fill = fix_offsets(df)
    df_fill = ffill(df_fill)
    df_fill['ask1_bid1_spread'] = df_fill['ask_price1'] / df_fill['bid_price1'] - 1
    
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return_new)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return_new)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return_new)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return_new)
    # Calculate wap balance
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    #add
    df['ask1_bid1_spread'] = df['ask_price1'] / df['bid_price1'] - 1
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    
    # Dict for aggregations
    create_feature_dict = {
        'log_return1': [realized_volatility, realized_quarticity, realized_1, np.std, np.sum],
        'log_return2': [realized_volatility, realized_quarticity, realized_1, np.std, np.sum],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'wap_balance': [np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'price_spread': [np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'ask1_bid1_spread': [np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'wap_balance': [np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std]
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict, seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility]
    }
    
    def aggregateBookData(df_, interval):
        df = df_.copy()
        df['interval'] = df['seconds_in_bucket'] // interval

        df_agg = df.groupby(['time_id', 'interval']).agg(
                            ask1_bid1_spread_avg = pd.NamedAgg(column='ask1_bid1_spread', aggfunc=np.mean) ).reset_index()

        df_wide = pd.pivot_table(df_agg, values=['ask1_bid1_spread_avg'], 
                                         index='time_id', columns='interval').reset_index().fillna(0)
        df_wide.columns = ['_'.join(str(e) for e in col) for col in df_wide.columns]
        df_wide = df_wide.add_suffix(f'_{interval}s_wide').rename(columns={f'time_id__{interval}s_wide' : 'time_id'})

        return df_wide
    
    df_interval = aggregateBookData(df_fill, 30)
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    df_interval['row_id'] = df_interval['time_id'].apply(lambda x: f'{stock_id}-{x}')
    df_interval.drop(['time_id'], axis = 1, inplace = True)
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    df_prices['row_id'] = df_prices['time_id'].apply(lambda x: f'{stock_id}-{x}')
    df_prices.drop(['time_id'], axis = 1, inplace = True)
    df_feature = pd.merge(df_feature, df_prices, how='left', on='row_id')
    df_feature = pd.merge(df_feature, df_book_datar, how='left', on='row_id')
    df_feature = pd.merge(df_feature, df_interval, how='left', on='row_id')
    
    return df_feature

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df_trade_datar = getFeaturesFromTradeData(df)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return_new)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility, realized_1],
        'seconds_in_bucket':[count_unique],
        'size':[np.std, np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min]
    }
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'order_count':[np.sum],
        'size':[np.sum]
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        
        # vol vars
        
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    
    df_lr = pd.DataFrame(lis)
        
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    df_feature = df_feature.merge(df_trade_datar, on = ['time_id'], how = 'left')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500', 'time_id__400', 'time_id__300', 'time_id__200', 'time_id__100', 'time_id'], axis = 1, inplace = True)
    
    
    df_feature['vega'] = np.sum(df['price'].values)/(df_feature['log_return_realized_volatility']+SMALL_F)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    # Get realized volatility columns
    vol_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility']

    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the stock id
    df_time_id = df.groupby(['time_id'])[vol_cols].agg(['mean', 'std', 'max', 'min']).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df
    
# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, is_train = True):
    
    # Parrallel for loop
    def for_joblib(stock_id):
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # Test
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
    
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        # Return the merge dataframe
        return df_tmp
    
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 10)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    #df = [for_joblib(stock_id) for stock_id in list_stock_ids]
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

## construct features across dataset

In [ ]:
def getDataFromTransformedDataFx(df, prefix=''):
    used_cols = ['log_return1_sum',
     'log_return1_realized_volatility',
     'log_return1_realized_quarticity',
     'log_return1_realized_1',
     'trade_log_return_realized_volatility',
     'price_spread_sum',
     'trade_abs_diff_v',
     'trade_p_size_mean']
    for c in used_cols:
        df.loc[:, prefix+c+'_rate'] = df.loc[:,c] / (np.mean(df.loc[:,c]) + SMALL_F)
        df.loc[:, prefix+c+'_diff'] = df.loc[:,c] - (np.mean(df.loc[:,c]) )

    return df

def getDataFromTransformedData(df):
    tmp1 = df.groupby(['stock_id']).apply(getDataFromTransformedDataFx, prefix='stock_id_').reset_index(drop=True)
    cols_tmp1 = [x for x in tmp1.columns if 'stock_id_' in x]
    tmp1['row_id'] = tmp1['stock_id'].astype(str) + '-' + tmp1['time_id'].astype(str)
    cols_tmp1.append('row_id')
    tmp2 = df.groupby(['time_id']).apply(getDataFromTransformedDataFx, prefix='time_id_').reset_index(drop=True)
    cols_tmp2 = [x for x in tmp2.columns if 'time_id_' in x]
    tmp2['row_id'] = tmp2['stock_id'].astype(str) + '-' + tmp2['time_id'].astype(str)
    cols_tmp2.append('row_id')

    df = df.merge(tmp1.loc[:,cols_tmp1], on = ['row_id'], how = 'left')
    df = df.merge(tmp2.loc[:,cols_tmp2], on = ['row_id'], how = 'left')
    
    print(df.shape)

    return df

## Model Definition

## Prepare test data

In [ ]:
train, test = read_train_test()
# Get unique stock ids 
#train_stock_ids = train['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
#train_ = preprocessor(train_stock_ids, is_train = True)
#train = train.merge(train_, on = ['row_id'], how = 'left')

# Get unique stock ids 
test_stock_ids = test['stock_id'].unique()
# Preprocess them using Parallel and our single stock id functions
test_ = preprocessor(test_stock_ids, is_train = False)
test = test.merge(test_, on = ['row_id'], how = 'left')

In [ ]:
train = pd.read_csv('../input/optivertraincsvshigeriatmp/train_v23.csv', index_col=0)

In [ ]:
train.drop(['trade_vega'], axis=1, inplace=True)
test.drop(['trade_vega'], axis=1, inplace=True)

In [ ]:
train = getDataFromTransformedData(train)
test = getDataFromTransformedData(test)

In [ ]:
# Get group stats of time_id and stock_id
train = get_time_stock(train)
test = get_time_stock(test)

In [ ]:
train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
train['size_tau2_d'] = np.sqrt( 0.33/ train['trade_order_count_sum'] ) - train['size_tau2']
test['size_tau2_d'] = np.sqrt( 0.33/ test['trade_order_count_sum'] ) - test['size_tau2']

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.to_csv('train_tmp.csv')
test.to_csv('test_tmp.csv')

In [ ]:
target_name = 'target2'
scores_folds = {}

In [ ]:
def root_mean_squared_per_error(y_true, y_pred):
         return K.sqrt(K.mean(K.square( (y_true - y_pred)/ y_true )))
    
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=20, verbose=0,
    mode='min',restore_best_weights=True)

plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.2, patience=7, verbose=0,
    mode='min')

In [ ]:
# making agg features

train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=42).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

for i, ind in enumerate(l):
    print(ind)
    newDf = train.loc[train['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(i)+'c1'
    mat.append ( newDf )
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(i)+'c1'
    matTest.append ( newDf )
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()

In [ ]:
matTest = []
mat = []
kmeans = []

mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])

mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

In [ ]:
nnn = ['time_id',
     'log_return1_realized_volatility_0c1',
     'log_return1_realized_volatility_1c1',
     'log_return1_realized_volatility_3c1',
     'log_return1_realized_volatility_4c1',
     'log_return1_realized_volatility_5c1',
     'total_volume_sum_0c1',
     'total_volume_sum_1c1', 
     'total_volume_sum_3c1',
     'total_volume_sum_4c1', 
     'total_volume_sum_5c1',
     'trade_order_count_sum_0c1',
     'trade_order_count_sum_1c1',
     'trade_order_count_sum_3c1',
     'trade_order_count_sum_4c1',
     'trade_order_count_sum_5c1',
     'bid_spread_sum_0c1',
     'bid_spread_sum_1c1',
     'bid_spread_sum_3c1',
     'bid_spread_sum_4c1',
     'bid_spread_sum_5c1',       
     'ask_spread_sum_0c1',
     'ask_spread_sum_1c1',
     'ask_spread_sum_3c1',
     'ask_spread_sum_4c1',
     'ask_spread_sum_5c1',
     'ask1_bid1_spread_sum_0c1',
     'ask1_bid1_spread_sum_1c1',
     'ask1_bid1_spread_sum_3c1',
     'ask1_bid1_spread_sum_4c1',
     'ask1_bid1_spread_sum_5c1',
     'size_tau2_0c1',
     'size_tau2_1c1',
     'size_tau2_3c1',
     'size_tau2_4c1',
     'size_tau2_5c1'] 

In [ ]:
train = pd.merge(train,mat1[nnn],how='left',on='time_id')

In [ ]:
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

In [ ]:
del mat1,mat2
gc.collect()

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

def train_and_evaluate(train, test):
    # Hyperparammeters (optimized)
    seed = 42
    params = {
            'learning_rate': 0.1,        
            'lambda_l1': 2,
            'lambda_l2': 7,
            'num_leaves': 800,
            'min_sum_hessian_in_leaf': 20,
            'feature_fraction': 0.8,
            'feature_fraction_bynode': 0.8,
            'bagging_fraction': 0.9,
            'bagging_freq': 42,
            'min_data_in_leaf': 700,
            'max_depth': 4,
            'seed': seed,
            'feature_fraction_seed': seed,
            'bagging_seed': seed,
            'drop_seed': seed,
            'data_random_seed': seed,
            'objective': 'rmse',
            'boosting': 'gbdt',
            'verbosity': -1,
            'n_jobs':-1,
            'device': 'gpu'
        }
    
    # Split features and target
    groups = train['time_id'].astype(int)
    x = train.drop(['row_id', 'target', 'time_id'], axis = 1)
    y = train['target']
    x_test = test.drop(['row_id', 'time_id'], axis = 1)
    # Transform stock id to a numeric value
    x['stock_id'] = x['stock_id'].astype(int)
    x_test['stock_id'] = x_test['stock_id'].astype(int)
    # Create out of folds array
    oof_predictions = np.zeros(x.shape[0])
    # Create test array to store predictions
    test_predictions = np.zeros(x_test.shape[0])
    # Create a KFold object
    kfold = GroupKFold(n_splits = 10)
    # Iterate through each fold
    for fold, (trn_ind, val_ind) in enumerate(kfold.split(x, y, groups)):
        print(f'Training fold {fold + 1}')
        x_train, x_val = x.iloc[trn_ind], x.iloc[val_ind]
        y_train, y_val = y.iloc[trn_ind], y.iloc[val_ind]
        print(x_train.shape, x_val.shape, x.shape)
        # Root mean squared percentage error weights
        train_weights = 1 / np.square(y_train)
        val_weights = 1 / np.square(y_val)
        train_dataset = lgb.Dataset(x_train, y_train, weight = train_weights, categorical_feature = ['stock_id'])
        val_dataset = lgb.Dataset(x_val, y_val, weight = val_weights, categorical_feature = ['stock_id'])
        model = lgb.train(params = params, 
                          train_set = train_dataset, 
                          valid_sets = [train_dataset, val_dataset], 
                          num_boost_round = 10000, 
                          early_stopping_rounds = 100, 
                          verbose_eval = 200,
                          feval = feval_rmspe)
        # Add predictions to the out of folds array
        oof_predictions[val_ind] = model.predict(x_val, num_iteration=model.best_iteration)
        # Predict the test set
        test_predictions += model.predict(x_test, num_iteration=model.best_iteration) / 10
        
    rmspe_score = rmspe(y, oof_predictions)
    print(f'Our out of folds RMSPE is {rmspe_score}')
    # Return test predictions
    return test_predictions

In [ ]:
# Traing and evaluate
test_predictions = train_and_evaluate(train, test)

In [ ]:
def rmspe(y_true, y_pred):
    # Function to calculate the root mean squared percentage error
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

class RMSPE(Metric):
    def __init__(self):
        self._name = "rmspe"
        self._maximize = False

    def __call__(self, y_true, y_score):
        
        return np.sqrt(np.mean(np.square((y_true - y_score) / y_true)))
    
def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 )).clone()

In [ ]:
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)

fill_0_features = ['log_return1_realized_volatility_500',
'log_return2_realized_volatility_500',
'log_return3_realized_volatility_500',
'log_return4_realized_volatility_500',
'trade_log_return_realized_volatility',
'trade_log_return_realized_1',
'trade_seconds_in_bucket_count_unique',
'trade_size_sum',
'trade_size_amax',
'trade_size_amin',
'trade_order_count_sum',
'trade_order_count_amax',
'trade_amount_sum',
'trade_amount_amax',
'trade_amount_amin',
'trade_energy_v',
'trade_log_return_realized_volatility_500',
'trade_seconds_in_bucket_count_unique_500',
'trade_order_count_sum_500',
'trade_size_sum_500',
'trade_log_return_realized_volatility_400',
'trade_seconds_in_bucket_count_unique_400',
'trade_order_count_sum_400',
'trade_size_sum_400',
'trade_log_return_realized_volatility_300',
'trade_seconds_in_bucket_count_unique_300',
'trade_order_count_sum_300',
'trade_size_sum_300',
'trade_log_return_realized_volatility_200',
'trade_seconds_in_bucket_count_unique_200',
'trade_order_count_sum_200',
'trade_size_sum_200',
'trade_log_return_realized_volatility_100',
'trade_seconds_in_bucket_count_unique_100',
'trade_order_count_sum_100',
'trade_size_sum_100',
'trade_p_price_count',
'stock_id_trade_log_return_realized_volatility_rate',
'stock_id_trade_log_return_realized_volatility_diff',
'time_id_trade_log_return_realized_volatility_rate',
'time_id_trade_log_return_realized_volatility_diff']

train[fill_0_features] = train[fill_0_features].fillna(0)
test[fill_0_features] = test[fill_0_features].fillna(0)

features = [col for col in list(train) if col not in ['time_id','target','row_id', 'stock_id']]
train[features] = train[features].fillna(train[features].mean())
test[features] = test[features].fillna(train[features].mean())

In [ ]:
nunique = train[features].nunique()
types = train[features].dtypes

categorical_columns = []
categorical_dims =  {}

qt_lis = []

for col in features:
    if  col == 'stock_id':
        l_enc = LabelEncoder()
        train[col] = l_enc.fit_transform(train[col].values)
        test[col] = l_enc.transform(test[col].values)
        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        scaler = StandardScaler()
        train[col] = scaler.fit_transform(train[col].values.reshape(-1, 1))
        test[col] = scaler.transform(test[col].values.reshape(-1, 1))
        
train[features] = train[features].fillna(train[features].mean())
test[features] = test[features].fillna(train[features].mean())

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

In [ ]:
tabnet_params = dict(
    cat_idxs=cat_idxs,
    cat_dims=cat_dims,
    cat_emb_dim=1,
    n_d = 16,
    n_a = 16,
    n_steps = 2,
    gamma = 2,
    n_independent = 2,
    n_shared = 2,
    lambda_sparse = 0,
    optimizer_fn = Adam,
    optimizer_params = dict(lr = (0.02)),
    mask_type = "entmax",
    scheduler_params = dict(T_0=200, T_mult=1, eta_min=1e-4, last_epoch=-1, verbose=False),
    scheduler_fn = CosineAnnealingWarmRestarts,
    seed = 42,
    verbose = 10   
)

In [ ]:
groups = train['time_id'].astype(int)
X = train.drop(['row_id', 'target', 'time_id'], axis = 1)
y = train['target']
X_test = test.drop(['time_id', 'row_id'], axis=1)
kfold = GroupKFold(n_splits = 5)
# Create out of folds array
oof_predictions = np.zeros((X.shape[0], 1))
test_predictions_tabnet = np.zeros(X_test.shape[0])

for fold, (trn_ind, val_ind) in enumerate(kfold.split(X, y, groups)):
    print(f'Training fold {fold + 1}')
    X_train, X_val = X.iloc[trn_ind].values, X.iloc[val_ind].values
    y_train, y_val = y.iloc[trn_ind].values.reshape(-1,1), y.iloc[val_ind].values.reshape(-1,1)


    clf = TabNetRegressor(**tabnet_params)
    clf.fit(
      X_train, y_train,
      eval_set=[(X_val, y_val)],
      max_epochs = 200,
      patience = 50,
      batch_size = 1024*20, 
      virtual_batch_size = 128*20,
      num_workers = 4,
      drop_last = False,
      eval_metric=[RMSPE],
      loss_fn=RMSPELoss
      )
    
    saving_path_name = f"./fold{fold}"
    saved_filepath = clf.save_model(saving_path_name)
    
    oof_predictions[val_ind] = clf.predict(X_val)
    test_predictions_tabnet += clf.predict(X_test.values).flatten()/5
    
print(f'OOF score across folds: {rmspe(y, oof_predictions.flatten())}')

In [ ]:
del train, test
gc.collect()

In [ ]:
train = pd.read_csv('./train_tmp.csv', index_col=0)
test = pd.read_csv('./test_tmp.csv', index_col=0)

In [ ]:
colNames = list(train)

colNames.remove('time_id')
colNames.remove('target')
colNames.remove('row_id')
colNames.remove('stock_id')

In [ ]:
train.replace([np.inf, -np.inf], np.nan,inplace=True)
test.replace([np.inf, -np.inf], np.nan,inplace=True)
qt_train = []

for col in colNames:
    #print(col)
    qt = QuantileTransformer(random_state=42,n_quantiles=4000, output_distribution='normal')
    train[col] = qt.fit_transform(train[[col]])
    test[col] = qt.transform(test[[col]])    
    qt_train.append(qt)

In [ ]:
# making agg features

train_p = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')

corr = train_p.corr()

ids = corr.index

kmeans = KMeans(n_clusters=7, random_state=42).fit(corr.values)
print(kmeans.labels_)

l = []
for n in range(7):
    l.append ( [ (x-1) for x in ( (ids+1)*(kmeans.labels_ == n)) if x > 0] )
    

mat = []
matTest = []

n = 0
for i, ind in enumerate(l):
    print(ind)
    newDf = train.loc[train['stock_id'].isin(ind) ]
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(i)+'c1'
    mat.append ( newDf )
    
    newDf = test.loc[test['stock_id'].isin(ind) ]    
    newDf = newDf.groupby(['time_id']).agg(np.nanmean)
    newDf.loc[:,'stock_id'] = str(i)+'c1'
    matTest.append ( newDf )
    
mat1 = pd.concat(mat).reset_index()
mat1.drop(columns=['target'],inplace=True)

mat2 = pd.concat(matTest).reset_index()

matTest = []
mat = []
kmeans = []

mat2 = pd.concat([mat2,mat1.loc[mat1.time_id==5]])

mat1 = mat1.pivot(index='time_id', columns='stock_id')
mat1.columns = ["_".join(x) for x in mat1.columns.ravel()]
mat1.reset_index(inplace=True)

mat2 = mat2.pivot(index='time_id', columns='stock_id')
mat2.columns = ["_".join(x) for x in mat2.columns.ravel()]
mat2.reset_index(inplace=True)

In [ ]:
train = pd.merge(train,mat1[nnn],how='left',on='time_id')
test = pd.merge(test,mat2[nnn],how='left',on='time_id')

In [ ]:
test['target1'] = test_predictions
test['target3'] = test_predictions_tabnet

In [ ]:
# kfold based on the knn++ algorithm

out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

#out_train[out_train.isna().any(axis=1)]
out_train = out_train.fillna(out_train.mean())
out_train.head()

# code to add the just the read data after first execution

# data separation based on knn ++
nfolds = 10 # number of folds
index = []
totDist = []
values = []
# generates a matriz with the values of 
mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0]/nfolds) # number of individuals

# adds index in the last column
mat = np.c_[mat,np.arange(mat.shape[0])]


lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)

lineNumber = np.sort(lineNumber)[::-1]

for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# saves index
for n in range(nfolds):
    
    values.append([lineNumber[n]])    


s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
         # saves the values of index           

        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [ ]:
#https://bignerdranch.com/blog/implementing-swish-activation-function-in-keras/
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})

In [ ]:
hidden_units = (64, 32, 16)
stock_embedding_size = 24

cat_data = train['stock_id']

def base_model():
    
    # Each instance will consist of two inputs: a single user id, and a single movie id
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(222,), name='num_data')

    #embedding, flatenning and concatenating
    x = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                           input_length=1, name='stock_embedding')(stock_id_input)
    y = tf.expand_dims(num_input, axis=1)
    x = keras.layers.Concatenate(axis=-1)([x, y])
    x = keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(256, return_sequences=True))(x)
    x = keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNGRU(128, return_sequences=True))(x)
    avg_pool = keras.layers.GlobalAveragePooling1D()(x)
    max_pool = keras.layers.GlobalMaxPooling1D()(x)
    conc = keras.layers.concatenate([avg_pool, max_pool])
    out = keras.layers.Flatten()(conc)
    
    # Add one or more hidden layers
    for n_hidden in hidden_units:

        out = keras.layers.Dense(n_hidden, activation='swish')(out)
        

    #out = keras.layers.Concatenate()([out, num_input])

    # A single output: our predicted rating
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
    inputs = [stock_id_input, num_input],
    outputs = out,
    )
    
    return model

In [ ]:
model_name = 'NN'
pred_name = 'pred_{}'.format(model_name)

n_folds = 10
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
scores_folds[model_name] = []
counter = 1

features_to_consider = list(train)

features_to_consider.remove('time_id')
features_to_consider.remove('target')
features_to_consider.remove('row_id')
try:
    features_to_consider.remove('pred_NN')
except:
    pass

train[fill_0_features] = train[fill_0_features].fillna(0)
test[fill_0_features] = test[fill_0_features].fillna(0)

train[features_to_consider] = train[features_to_consider].fillna(train[features_to_consider].mean())
test[features_to_consider] = test[features_to_consider].fillna(train[features_to_consider].mean())

train[pred_name] = 0
test['target2'] = 0

In [ ]:
for n_count in range(n_folds):
    print('CV {}/{}'.format(counter, n_folds))
    
    indexes = np.arange(nfolds).astype(int)    
    indexes = np.delete(indexes,obj=n_count, axis=0) 
    
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]],values[indexes[4]],values[indexes[5]],values[indexes[6]],values[indexes[7]], values[indexes[8]]]
    
    X_train = train.loc[train.time_id.isin(indexes), features_to_consider]
    y_train = train.loc[train.time_id.isin(indexes), 'target']
    X_test = train.loc[train.time_id.isin(values[n_count]), features_to_consider]
    y_test = train.loc[train.time_id.isin(values[n_count]), 'target']
    
    #############################################################################################
    # NN
    #############################################################################################
    
    model = base_model()
    
    model.compile(
        keras.optimizers.Adam(learning_rate=0.006), #0.005
        loss=root_mean_squared_per_error
    )
    
    try:
        features_to_consider.remove('stock_id')
    except:
        pass
    
    num_data = X_train[features_to_consider]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))         
    num_data = scaler.fit_transform(num_data.values)    
    
    cat_data = X_train['stock_id'] 
    target =  y_train
    
    num_data_test = X_test[features_to_consider]
    num_data_test = scaler.transform(num_data_test.values)
    cat_data_test = X_test['stock_id']

    model.fit([cat_data, num_data], 
              target,               
              batch_size=2048,
              epochs=200, # 1000
              validation_data=([cat_data_test, num_data_test], y_test),
              callbacks=[es, plateau],
              validation_batch_size=len(y_test),
              shuffle=True,
             verbose = 1)

    preds = model.predict([cat_data_test, num_data_test]).reshape(1,-1)[0]
    
    score = round(rmspe(y_true = y_test, y_pred = preds),5)
    print('Fold {} {}: {}'.format(counter, model_name, score))
    scores_folds[model_name].append(score)
    
    tt = scaler.transform(test[features_to_consider].values)
    train.loc[train.time_id.isin(values[n_count]), pred_name] += preds
    test[target_name] += model.predict([test['stock_id'], tt]).reshape(1,-1)[0].clip(0,1e10)
    #test[target_name] += model.predict([test['stock_id'], test[features_to_consider]]).reshape(1,-1)[0].clip(0,1e10)
       
    counter += 1
    features_to_consider.append('stock_id')
    del model
    gc.collect()

In [ ]:
test[target_name] = test[target_name] / n_folds
score = round(rmspe(y_true = train['target'].values, y_pred = train[pred_name].values),5)
print('RMSPE {}: {} - Folds: {}'.format(model_name, score, scores_folds[model_name]))

In [ ]:
test['target'] = test['target1'] * 0.2 + test['target2'] * 0.4 + test['target3'] * 0.4

test[['row_id', 'target']].to_csv('submission.csv',index = False)

In [ ]:
test[['row_id', 'target', 'target1', 'target2', 'target3']].head(2)